In [0]:
!pip install tensorflow==2.0.0

import zipfile, os
import matplotlib.pyplot as plt
from shutil import rmtree
from datetime import datetime
from google.colab import files
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [0]:
print(tf.__version__)

In [0]:
optimizer = tf.keras.optimizers.Adam(lr=0.001)
regularizer = tf.keras.regularizers.l2(0.01)
input_shape = (150, 150, 3)
batch_size = 32
last_layer_to_select = 'mixed7'
early_stopping_patience = 10
epochs = 100
dropout_rate = 0.4
dense_units = [256]
dense_activation = 'relu'

In [0]:
dataset_zip_file_path = '/content/dataset.zip'
if not os.path.exists(dataset_zip_file_path):
  files.upload()

In [0]:
dataset_extracted_path = '/tmp/dataset'
rmtree(dataset_extracted_path, ignore_errors=True)
zip_file = zipfile.ZipFile(dataset_zip_file_path, 'r')
zip_file.extractall('/tmp')
zip_file.close()

In [0]:
training_path = os.path.join(dataset_extracted_path, 'training')
validation_path = os.path.join(dataset_extracted_path, 'validation')
testing_path = os.path.join(dataset_extracted_path, 'testing')

In [0]:
print(len(os.listdir(os.path.join(training_path, 'class1'))))
print(len(os.listdir(os.path.join(training_path, 'class2'))))
print(len(os.listdir(os.path.join(validation_path, 'class1'))))
print(len(os.listdir(os.path.join(validation_path, 'class2'))))
print(len(os.listdir(os.path.join(testing_path, 'class1'))))
print(len(os.listdir(os.path.join(testing_path, 'class2'))))

In [0]:
training_data_generator = ImageDataGenerator(
  preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,
  rotation_range=40,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode='nearest')

validation_data_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)

testing_data_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input)

training_generator = training_data_generator.flow_from_directory(
  training_path,
  target_size=(input_shape[0], input_shape[1]),
  batch_size=batch_size,
  shuffle=True,
  class_mode='binary')

validation_generator = validation_data_generator.flow_from_directory(
  validation_path,
  target_size=(input_shape[0], input_shape[1]),
  batch_size=batch_size,
  shuffle=True,
  class_mode='binary')

testing_generator = testing_data_generator.flow_from_directory(
  testing_path,
  target_size=(input_shape[0], input_shape[1]),
  batch_size=batch_size,
  shuffle=False,
  class_mode='binary')

In [0]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [0]:
inception_weights_file_path = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = input_shape, include_top = False, weights = None)

pre_trained_model.load_weights(inception_weights_file_path)

for layer in pre_trained_model.layers: layer.trainable = False

last_layer = pre_trained_model.get_layer(last_layer_to_select)

x = last_layer.output
x = layers.Flatten() (x)
for units in dense_units:
  x = layers.Dense(units, activation=dense_activation, kernel_regularizer=regularizer) (x)
  x = layers.Dropout(dropout_rate) (x)
x = layers.Dense (1, activation='sigmoid') (x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])

model.summary()

In [0]:
STEP_SIZE_TRAINING = training_generator.n // training_generator.batch_size
STEP_SIZE_VALIDATION = validation_generator.n // validation_generator.batch_size
STEP_SIZE_TESTING = testing_generator.n // testing_generator.batch_size

print(STEP_SIZE_TRAINING)
print(STEP_SIZE_VALIDATION)
print(STEP_SIZE_TESTING)

In [0]:
model_file_path = os.path.join('/content', datetime.now().strftime("model-snapshot-%Y-%m-%d-%H-%M-%S") + '.hdf5')

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=early_stopping_patience, restore_best_weights=True, verbose=1)

check_pointer = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath=model_file_path, save_best_model=True, verbose=1)

history = model.fit_generator(
  generator=training_generator,
  steps_per_epoch=STEP_SIZE_TRAINING,  
  epochs=epochs,
  validation_data=validation_generator,
  validation_steps=STEP_SIZE_VALIDATION,
  callbacks=[early_stopping, check_pointer],
  verbose=1)

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.figure()
plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training / Validation Accuracies')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training / Validation Loss')
plt.legend()

plt.show()

In [0]:
print(f'Testing Loss, Testing Accuracy: {model.evaluate_generator(generator=testing_generator, steps=STEP_SIZE_TESTING)}')
print(f'Training Loss, Training Accuracy: {model.evaluate_generator(generator=training_generator, steps=STEP_SIZE_TRAINING)}')
print(f'Validation Loss, Validation Accuracy: {model.evaluate_generator(generator=validation_generator, steps=STEP_SIZE_VALIDATION)}')

In [0]:
!rm --force /content/*.jpg /content/*.png

label_names = { v: k for k, v in training_generator.class_indices.items() }

uploaded_files = files.upload()

for file_name in uploaded_files.keys():
  file_path = os.path.join('/content', file_name)

  img = image.load_img(file_path, target_size=(input_shape[0], input_shape[1]))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.inception_v3.preprocess_input(x)

  classes = model.predict(x)

  if classes[0] > 0.5:
    print(f"[{file_name}] is predicted as [{label_names[1]}]")
  else:
    print(f"[{file_name}] is predicted as [{label_names[0]}]")